# Overview

This notebook shows how to train and test a neural network on the BELLA data.

In order to run this notebook, you need to produce first produce CSV files for the training and testing data. (See the folder `experimental data`.)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import transformer

from Neural_Net_Classes import NN as NN
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F

<h2>Loading Split Experimental Data</h2>

In [ ]:
# Load the CSV file
sim_training_set_df = pd.read_csv('simulation_data.csv')

# Access the arrays
sim_z_training_set = sim_training_set_df['z_target (m)'].values
sim_TOD_training_set = sim_training_set_df['TOD (s^3)'].values
sim_protons_training_set = sim_training_set_df['n_protons (1/sr)'].values

<h2>Visualizing Split Experimental Datasets</h2>

In [ ]:
plt.clf()
ax = plt.figure().add_subplot(projection='3d')

ax.scatter( sim_TOD_training_set, sim_z_training_set, sim_protons_training_set, c='r',alpha=0.3, label='Training Set')
#ax.scatter( TOD_test_set, z_test_set,protons_test_set, c='b', alpha=0.3, label='Testing Set')
ax.view_init(elev=40., azim=40)
plt.xlabel('TOD')
plt.ylabel('z_target')
plt.legend()

<h2>Normalizing Data</h2>

In [ ]:
#Setting Bounds
z_bounds = torch.tensor([-150e-6, 150e-6])
TOD_bounds = torch.tensor([-80e-41, 80e-41])
protons_bounds = torch.tensor([7e7, 1e11])

#define transformers
sim_transformer_z = transformer.Transformer(z_bounds.reshape(2,1), transform_type = 'normalize')
sim_transformer_TOD = transformer.Transformer(TOD_bounds.reshape(2,1), transform_type = 'normalize')
sim_transformer_protons = transformer.Transformer(protons_bounds.reshape(2,1), transform_type = 'normalize')

#Full normalization process
def normalization(array, transformer):
    array = np.array(array).reshape(-1,1)
    array = torch.tensor(array)
    norm = transformer.forward(array)
    return norm


In [ ]:
#Normalize datasets 1
norm_sim_z_train_set = normalization(sim_z_training_set, sim_transformer_z)
norm_sim_TOD_train_set = normalization(sim_TOD_training_set, sim_transformer_TOD)
norm_sim_protons_train_set = normalization(sim_protons_training_set, sim_transformer_protons)


<h2>Visualizing Normalized Data</h2>

In [ ]:
# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for training set
ax.scatter(norm_sim_TOD_train_set, norm_sim_z_train_set, norm_sim_protons_train_set, label='Sim Training Set', alpha=0.7)
# Scatter plot for testing set
#ax.scatter(norm_TOD_test_set, norm_z_test_set, norm_protons_test_set, label='Test Set', alpha=0.7)
ax.view_init(elev=40., azim=40)
# Set labels and title
ax.set_xlabel('Normalized TOD')
ax.set_ylabel('Normalized Z')
ax.set_zlabel('Normalized Protons')

# Add legend
ax.legend()
# Show plot
plt.show()

<h1>Neural Network Framework</h1>

<h2>Build and Train Neural Networks on simulation data</h2>

In [ ]:
sim_net = NN(learning_rate=0.0001)
sim_net.train_model(norm_sim_z_train_set, norm_sim_TOD_train_set, norm_sim_protons_train_set,num_epochs=10000)
sim_net.plot_loss()
#net.test_model(norm_z_test_set, norm_TOD_test_set, norm_protons_test_set)

In [ ]:
train_predictions = sim_net.predict(norm_sim_z_train_set, norm_sim_TOD_train_set)


<h2>Plotting Predictions from NN trained on simulations </h2>

In [ ]:
fig, ax = plt.subplots()

ax.scatter(norm_sim_z_train_set, norm_sim_protons_train_set, label='Training Set 1')


ax.scatter(train_predictions['Z_target'], train_predictions['predictions'], label='predictions', s=50, facecolors='none', edgecolors='r')

plt.title("n_protons predictions")
plt.xlabel('z_target (m)')
plt.ylabel('Number of protons (1/sr)')
plt.legend()

In [ ]:
# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for training set
ax.scatter(norm_sim_TOD_train_set, norm_sim_z_train_set, norm_sim_protons_train_set, label='Training Set 1', alpha=0.7)
#ax.scatter(norm_TOD_test_set, norm_z_test_set, norm_protons_test_set, label='Test Set 1', alpha=0.7)

ax.scatter(train_predictions['TOD'], train_predictions['Z_target'], train_predictions['predictions'], label='predictions 1', s=50, facecolors='none', edgecolors='r')
#ax.scatter(test_predictions['TOD'], test_predictions['Z_target'], test_predictions['predictions'], s=50, facecolors='none', edgecolors='r')

ax.view_init(elev=40., azim=40)
# Set labels and title
ax.set_title('Simulation Data v Predictions')
ax.set_xlabel('TOD')
ax.set_ylabel('z_target')
ax.set_zlabel('n Protons')

# Add legend
ax.legend()
# Show plot
plt.show()

<h2> Define NN that adds linear calibration parameters to output obtained from pre-trained base neural net </h2>

In [ ]:

class finetune_NN(NN):
    def __init__(self, base_model, hidden_size=20, 
                 learning_rate=0.001, patience=100, factor=0.5, threshold=1e-4):
        super(finetune_NN, self).__init__(hidden_size, learning_rate, patience, factor, threshold)
        
        # Copying weights from base_model to this model
        self.hidden1.load_state_dict(base_model.hidden1.state_dict())
        self.hidden2.load_state_dict(base_model.hidden2.state_dict())
        self.hidden3.load_state_dict(base_model.hidden3.state_dict())
        self.hidden4.load_state_dict(base_model.hidden4.state_dict())
        self.hidden5.load_state_dict(base_model.hidden5.state_dict())        
        self.output.load_state_dict(base_model.output.state_dict())
        # This is the weight and bias to be tuned on the output obtained from base model
        self.w2 =torch.nn.Parameter(torch.rand(1))
        self.b2 = torch.nn.Parameter(torch.rand(1))
        self.relu = nn.ReLU()
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.scheduler = ReduceLROnPlateau(self.optimizer, 'min', 
                                           factor=factor, patience=patience, threshold=threshold)
        
    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.relu(self.hidden3(x))
        x = self.relu(self.hidden4(x))
        x = self.relu(self.hidden5(x))
        x = self.output(x)
        x = x * self.w2 + self.b2
        
        return x
    
    def train_model(self, x_train, y_train, z_train, num_epochs=1500):
        #print(self.parameters())
        for param in self.hidden1.parameters():
            param.requires_grad = False
        for param in self.hidden2.parameters():
            param.requires_grad = False
        for param in self.hidden3.parameters():
            param.requires_grad = False
        for param in self.hidden4.parameters():
            param.requires_grad = False
        for param in self.hidden5.parameters():
            param.requires_grad = False
        for param in self.output.parameters():
            param.requires_grad = False           
  
        x_train = x_train.to(torch.float32)
        y_train = y_train.to(torch.float32)
        z_train = z_train.to(torch.float32)
        
        inputs = torch.cat((x_train, y_train), dim=1).to(torch.float32)
        self.train()
        for epoch in range(num_epochs):
            self.optimizer.zero_grad()

            outputs = self(inputs)
            loss = self.criterion(outputs, z_train)
            loss.backward()
            torch.autograd.set_detect_anomaly(True)            
            self.optimizer.step()

            current_loss = loss.item()
            self.loss_data['loss'].append(current_loss)
            self.loss_data['epoch_count'].append(epoch)
            self.scheduler.step(current_loss)

            if (epoch + 1) % (num_epochs/1000) == 0:
                print(f'Comb NN: Epoch [{epoch+1}/{num_epochs}], Loss:{loss.item():.6f}')
                print("w2,b2",self.w2,self.b2)

    def predict(self, x_values, y_values):
        '''
        args:
            tensor x_values
            tensor y_values
        returns:
            numpy array with predictions
        '''
        predictions = {
        'Z_target': x_values.tolist(),
        'TOD': y_values.tolist(),
        'predictions': []
        }
    
        inputs = torch.cat((x_values, y_values), dim=1).to(torch.float32)
        self.eval()
        with torch.no_grad():
            output = self(inputs)
            predictions['predictions'] = output.detach().numpy().tolist()
    
        return predictions


<h1> Load experimental data previously split into training and testing </h1>

In [ ]:
# Load the CSV file
expt_training_set_df = pd.read_csv('expt_training_set_1.csv')

# Access the arrays
expt_z_training_set = expt_training_set_df['z_target (m)'].values
expt_TOD_training_set = expt_training_set_df['TOD (s^3)'].values
expt_protons_training_set = expt_training_set_df['n_protons (1/sr)'].values

In [ ]:
# Load the CSV file
expt_test_set_df = pd.read_csv('expt_test_set_1.csv')

# Access the arrays
expt_z_test_set = expt_test_set_df['z_target (m)'].values
expt_TOD_test_set = expt_test_set_df['TOD (s^3)'].values
expt_protons_test_set = expt_test_set_df['n_protons (1/sr)'].values

<h2> Set bounds and normalize expt data </h2>

In [ ]:
#Setting Bounds


#define transformers
expt_transformer_z = transformer.Transformer(z_bounds.reshape(2,1), transform_type = 'normalize')
expt_transformer_TOD = transformer.Transformer(TOD_bounds.reshape(2,1), transform_type = 'normalize')
expt_transformer_protons = transformer.Transformer(protons_bounds.reshape(2,1), transform_type = 'normalize')

#Full normalization process
def normalization(array, transformer):
    array = np.array(array).reshape(-1,1)
    array = torch.tensor(array)
    norm = transformer.forward(array)
    return norm

In [ ]:
#Normalize datasets 1
expt_norm_z_train_set = normalization(expt_z_training_set, expt_transformer_z)
expt_norm_TOD_train_set = normalization(expt_TOD_training_set, expt_transformer_TOD)
expt_norm_protons_train_set = normalization(expt_protons_training_set, expt_transformer_protons)
expt_norm_z_test_set = normalization(expt_z_test_set, expt_transformer_z)
expt_norm_TOD_test_set = normalization(expt_TOD_test_set, expt_transformer_TOD)
expt_norm_protons_test_set = normalization(expt_protons_test_set, expt_transformer_protons)

<h2> Train the linear scaling parameters on the output, namely, w2, b2, on the training set of experimental data </h2>

In [ ]:



finetune_net = finetune_NN(sim_net,learning_rate=0.001)
finetune_net.train_model(expt_norm_z_train_set,expt_norm_TOD_train_set,expt_norm_protons_train_set,num_epochs=10000)


In [ ]:
### Plot the loss and print the values of the learned calibration parameters 
finetune_net.plot_loss()
for name, param in finetune_net.named_parameters():
    if param.requires_grad:
        print(name, param.data)


In [ ]:
#Predictions from calibrated NN on testing data from experiments
finetune_net_pred_expt_test = finetune_net.predict(expt_norm_z_test_set,expt_norm_TOD_test_set)

In [ ]:
#Predictions from calibrated NN on testing data from experiments
finetune_net_pred = finetune_net.predict(expt_norm_z_train_set,expt_norm_TOD_train_set)

In [ ]:
#Predictions from the base NN model (uncalibrated) on testing and training data from experiments
sim_net_pred_expt_train = sim_net.predict(expt_norm_z_train_set,expt_norm_TOD_train_set)
sim_net_pred_expt_test = sim_net.predict(expt_norm_z_test_set,expt_norm_TOD_test_set)

In [ ]:
#Visualize the data

fig, ax = plt.subplots(figsize=(10,8))

ax.scatter(expt_norm_z_train_set, expt_norm_protons_train_set, label='Training Set 1')
ax.scatter(expt_norm_z_test_set, expt_norm_protons_test_set, label='Test Set 1')
ax.scatter(norm_sim_z_train_set, norm_sim_protons_train_set, label='Sim Training Set 1')

ax.scatter(finetune_net_pred['Z_target'], finetune_net_pred['predictions'], label=' expt train predictions', s=50, facecolors='none', edgecolors='m')
ax.scatter(finetune_net_pred_expt_test['Z_target'], finetune_net_pred_expt_test['predictions'],label=' expt test predictions', s=50, facecolors='none', edgecolors='r')
ax.scatter(sim_net_pred_expt_train['Z_target'], sim_net_pred_expt_train['predictions'], label=' sim NN predictions for expt train', s=50, facecolors='none', edgecolors='cyan')
ax.scatter(sim_net_pred_expt_test['Z_target'], sim_net_pred_expt_test['predictions'], label=' sim NN predictions for expt train', s=50, facecolors='none', edgecolors='blue')
plt.title("n_protons predictions")
plt.xlabel('z_target (m)')
plt.ylabel('Number of protons (1/sr)')
plt.legend()